In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import sys
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Circle
import seaborn as sns; 
from IPython.display import HTML

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn import svm
from sklearn.metrics import r2_score

In [3]:
#config parser
import configparser

sys.path.insert(0, '../Common/')
from AISDataManager import AISDataManager
import Constants as c
import HMUtils as hMUtil
import TimeUtils as timeUtils

#MyConfig.INI stores all the run time constants
config = configparser.ConfigParser()
config.read('../MyConfig.INI')

from joblib import Parallel, delayed
import multiprocessing
aISDM = AISDataManager()
numCores = multiprocessing.cpu_count()

In [4]:
lonMin = (float)(config['REGEION']['LON_MIN'])
lonMax = (float)(config['REGEION']['LON_MAX'])

latMin = (float)(config['REGEION']['LAT_MIN'])
latMax = (float)(config['REGEION']['LAT_MAX'])


lonMin = (float)(-120.0)
lonMax = (float)(-119.0)

latMin = (float)(33.5)
latMax = (float)(34.5)

print(lonMin,latMin)
print(lonMax,latMax)

# increStep = (float)(config['HEATMAP']['INCR_STEP'])
# incrRes = (int)(config['HEATMAP']['INCR_RES'])
increStep = (float)(0.05)
incrRes = (int)(2)

-120.0 33.5
-119.0 34.5


In [5]:
heatMapGrid = hMUtil.generate_grid(lonMin, lonMax, latMin, latMax, increStep, incrRes)

In [6]:
boundaryArray = heatMapGrid[2]
horizontalAxis = heatMapGrid[0]
verticalAxis = heatMapGrid[1]

In [7]:
neighTPCount = []
for i in range(9):
    neighTPCount.append(np.zeros((horizontalAxis.shape[0] * verticalAxis.shape[0])))
    
tPCurrentSum = np.zeros((horizontalAxis.shape[0] * verticalAxis.shape[0]))

In [8]:
#this function will get Lon and LAT of neighbouring pixel from single dimension
#012
#345
#678
#where fourth is pixel of interest
def get_neighbour_lon_lat(pixelIdx,nearIdx):
    #access boundary array
    #boundary array is sequential in nature
    pixelLonMin = boundaryArray[pixelIdx][0]
    pixelLonMax = boundaryArray[pixelIdx][1]
    pixelLatMin = boundaryArray[pixelIdx][2]
    pixelLatMax = boundaryArray[pixelIdx][3]
    
    #offset for minimum
    lonDiffArr = [ -increStep, 0, increStep \
                 , -increStep, 0, increStep \
                 , -increStep, 0, increStep \
                ]

    latDiffArr = [ increStep, increStep, increStep \
                 , 0, 0, 0 \
                 , -increStep, -increStep, -increStep \
                ]
    
    retLonMin = pixelLonMin + lonDiffArr[nearIdx]
    retLonMax = pixelLonMax + lonDiffArr[nearIdx]
    
    retLatMin = pixelLatMin + latDiffArr[nearIdx]
    retLatMax = pixelLatMax + latDiffArr[nearIdx]
    
    retLonMin = np.around(retLonMin,incrRes)
    retLonMax = np.around(retLonMax,incrRes)
    
    retLatMin = np.around(retLatMin,incrRes)
    retLatMax = np.around(retLatMax,incrRes)
    return retLonMin, retLonMax, retLatMin, retLatMax

In [9]:
fileDir = "../Data/AIS_SB/Tenly/"
dirToStore = "../Data/AIS_SB/TenlyTP/"

In [10]:
# for grid in boundaryArray:
def parallel_grid_exec(grid,firstDF,secondDF):
    currentGrid = grid[4]
    #count number of vessels in a regeion
    rOILonMin = grid[0]
    rOILonMax = grid[1]
    rOILatMin = grid[2]
    rOILatMax = grid[3]

    #get the data for this rOI
    currentROIDF = aISDM.filter_based_on_lon_lat(firstDF,rOILonMin,rOILonMax,rOILatMin,rOILatMax)
    currVessels = aISDM.get_list_of_unique_mmsi(currentROIDF)

    countNeigbList = []
    for neighb in range(9):
        neihbLonMin, neihbLonMax, neihbLatMin, neihbLatMax = \
            get_neighbour_lon_lat(currentGrid,neighb)
        nextDF = aISDM.filter_based_on_lon_lat(secondDF,neihbLonMin,neihbLonMax,neihbLatMin,neihbLatMax)
        nextNeighbVessels = aISDM.get_list_of_unique_mmsi(nextDF)
        countNeigb = len(set(currVessels) & set(nextNeighbVessels))
        countNeigbList.append(countNeigb)
    return countNeigbList, len(currVessels), currentGrid

In [11]:
def clear_tm_matrix():
    for i in range(9):
        neighTPCount[i].fill(0)
    tPCurrentSum.fill(0)
    
def compute_transition_prob(firstNum):
    fileName1 = fileDir + str(firstNum) + '.csv'
    fileName2 = fileDir + str(firstNum+1) + '.csv'

    firstDF,_ = aISDM.load_data_from_csv(fileName1)
    secondDF,_ = aISDM.load_data_from_csv(fileName2)
    
    tPValues = Parallel(n_jobs=numCores, backend = 'multiprocessing', verbose=10)(delayed(parallel_grid_exec)(grid, firstDF, secondDF) for grid in boundaryArray)
    for tP in range(len(tPValues)):
        countNeigbList = tPValues[tP][0]
        currentGrid = tPValues[tP][2]
        for neighb in range(9):
            neighTPCount[neighb][currentGrid] = neighTPCount[neighb][currentGrid] + countNeigbList[neighb]
            
        tPCurrentSum[currentGrid] = tPCurrentSum[currentGrid] + tPValues[tP][1]

In [12]:
clear_tm_matrix()
compute_transition_prob(0)

[Parallel(n_jobs=8)]: Using backend MultiprocessingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Batch computation too fast (0.0219s.) Setting batch_size=18.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 400 out of 400 | elapsed:    0.9s finished


In [13]:
for i in range(neighTPCount[5].shape[0]):
    if(neighTPCount[5][i] > 0):
        print(neighTPCount[5][i])

1.0
1.0
1.0
1.0


In [14]:
for i in range(tPCurrentSum.shape[0]):
    if(tPCurrentSum[i] > 0):
        print(tPCurrentSum[i])

1.0
1.0
2.0
2.0
1.0
7.0
1.0
28.0
11.0
5.0
1.0
1.0
1.0
1.0
9.0


In [15]:
clear_tm_matrix()

In [16]:
for fileCounter in range(7200):
    compute_transition_prob(fileCounter)

[Parallel(n_jobs=8)]: Using backend MultiprocessingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Batch computation too fast (0.0208s.) Setting batch_size=18.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 400 out of 400 | elapsed:    0.9s finished
[Parallel(n_jobs=8)]: Using backend MultiprocessingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Batch computation too fast (0.0238s.) Setting batch_size=16.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 160 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 400 out of 400 | elapsed:    0.9s finished
[Parallel(n_jobs=8)]: Using backend MultiprocessingBackend with 8 concurrent workers.
[Parallel

[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 400 out of 400 | elapsed:    0.9s finished
[Parallel(n_jobs=8)]: Using backend MultiprocessingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Batch computation too fast (0.0349s.) Setting batch_size=10.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Batch computation too fast (0.1695s.) Setting batch_size=22.
[Parallel(n_jobs=8)]: Done 106 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 400 out of 400 | elapsed:    1.0s finished
[Parallel(n_jobs=8)]: Using backend MultiprocessingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Batch computation too fast (0.0206s.) Setting batch_size=18.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_

[Parallel(n_jobs=8)]: Done 160 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 400 out of 400 | elapsed:    0.9s finished
[Parallel(n_jobs=8)]: Using backend MultiprocessingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Batch computation too fast (0.0239s.) Setting batch_size=16.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 160 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 400 out of 400 | elapsed:    0.9s finished
[Parallel(n_jobs=8)]: Using backend MultiprocessingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Batch computation too fast (0.0364s.) Setting batch_size=10.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Batch comput

[Parallel(n_jobs=8)]: Done 400 out of 400 | elapsed:    0.9s finished
[Parallel(n_jobs=8)]: Using backend MultiprocessingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Batch computation too fast (0.0259s.) Setting batch_size=14.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 142 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 400 out of 400 | elapsed:    0.9s finished
[Parallel(n_jobs=8)]: Using backend MultiprocessingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Batch computation too fast (0.0214s.) Setting batch_size=18.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 400 out of 400 | elapsed:    0.9s finished
[Parallel(n_jobs=8)]: Usi

[Parallel(n_jobs=8)]: Using backend MultiprocessingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Batch computation too fast (0.0350s.) Setting batch_size=10.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Batch computation too fast (0.1712s.) Setting batch_size=22.
[Parallel(n_jobs=8)]: Done 106 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 400 out of 400 | elapsed:    1.1s finished
[Parallel(n_jobs=8)]: Using backend MultiprocessingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Batch computation too fast (0.0215s.) Setting batch_size=18.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 400 out of 400 | elapsed:    0.9s finished
[Parallel(n_

...

[Parallel(n_jobs=8)]: Done 160 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 400 out of 400 | elapsed:    0.9s finished
[Parallel(n_jobs=8)]: Using backend MultiprocessingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Batch computation too fast (0.0221s.) Setting batch_size=18.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 400 out of 400 | elapsed:    0.9s finished
[Parallel(n_jobs=8)]: Using backend MultiprocessingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Batch computation too fast (0.0284s.) Setting batch_size=14.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 142 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 400 out

[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 106 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 196 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 400 out of 400 | elapsed:    0.9s finished
[Parallel(n_jobs=8)]: Using backend MultiprocessingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Batch computation too fast (0.0455s.) Setting batch_size=8.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Batch computation too fast (0.1806s.) Setting batch_size=16.
[Parallel(n_jobs=8)]: Done  88 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 400 out of 400 | elapsed:    1.0s finished
[Parallel(n_jobs=8)]: Using backend MultiprocessingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Batch computa

[Parallel(n_jobs=8)]: Done 400 out of 400 | elapsed:    0.9s finished
[Parallel(n_jobs=8)]: Using backend MultiprocessingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Batch computation too fast (0.0376s.) Setting batch_size=10.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Batch computation too fast (0.1780s.) Setting batch_size=22.
[Parallel(n_jobs=8)]: Done 106 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 400 out of 400 | elapsed:    1.1s finished
[Parallel(n_jobs=8)]: Using backend MultiprocessingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Batch computation too fast (0.0217s.) Setting batch_size=18.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.1s
[Parallel(n_

[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 106 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 196 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 400 out of 400 | elapsed:    0.9s finished
[Parallel(n_jobs=8)]: Using backend MultiprocessingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Batch computation too fast (0.0688s.) Setting batch_size=4.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Batch computation too fast (0.1470s.) Setting batch_size=10.
[Parallel(n_jobs=8)]: Done  52 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 100 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 210 tasks      | elapsed:    0.6s
[Paralle

[Parallel(n_jobs=8)]: Done  88 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 180 out of 400 | elapsed:    0.7s remaining:    0.8s
[Parallel(n_jobs=8)]: Done 400 out of 400 | elapsed:    1.0s finished
[Parallel(n_jobs=8)]: Using backend MultiprocessingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Batch computation too fast (0.0265s.) Setting batch_size=14.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 142 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 400 out of 400 | elapsed:    1.0s finished


In [17]:
#destination directory
destFileDir = "../Data/AIS_SB/TenlyTP/"
for i in range(9):
    opFile = destFileDir + str(i)+'.npy'
    np.save(opFile, neighTPCount[i])
    

In [18]:
opFile = destFileDir + 'Sum.npy'
np.save(opFile, tPCurrentSum)

In [ ]:
# #heat map arrays
# n40HeatMap = np.zeros((xGrid.shape[0] * yGrid.shape[0]))
# n41HeatMap = np.zeros((xGrid.shape[0] * yGrid.shape[0]))
# n42HeatMap = np.zeros((xGrid.shape[0] * yGrid.shape[0]))
# n43HeatMap = np.zeros((xGrid.shape[0] * yGrid.shape[0]))
# n44HeatMap = np.zeros((xGrid.shape[0] * yGrid.shape[0]))
# n45HeatMap = np.zeros((xGrid.shape[0] * yGrid.shape[0]))
# n46HeatMap = np.zeros((xGrid.shape[0] * yGrid.shape[0]))
# n47HeatMap = np.zeros((xGrid.shape[0] * yGrid.shape[0]))
# n48HeatMap = np.zeros((xGrid.shape[0] * yGrid.shape[0]))
# n44Sum = np.zeros((xGrid.shape[0] * yGrid.shape[0]))
# print(n44Sum.shape)

# # fileDir = "../Data/AIS_0117_1217_31_M120_345_M117/Hourly/"
# fileDir = "../Data/AIS_0117_1217_31_M120_345_M117/HourlyDummy/"

# #iterate through CSV files
# for fileCounter in range(0,1):
# # for file in range(0,1):
#     #take sequential files
#     print(fileCounter)
#     fileName1 = fileDir + str(fileCounter) + '.csv'
#     fileName2 = fileDir + str(fileCounter+1) + '.csv'
#     firstDF,_ = aISDM.load_data_from_csv(fileName1)
#     secondDF,_ = aISDM.load_data_from_csv(fileName2)
#     #first read the file
#     for grid in boundaryArray:
# #         if(grid[4] == 743):
#         currentGrid = grid[4]
#         #count number of vessels in a regeion
#         rOILonMin = grid[0]
#         rOILonMax = grid[1]
#         rOILatMin = grid[2]
#         rOILatMax = grid[3]
# #             print(rOILonMin,rOILonMax,rOILatMin,rOILatMax)
#         #get the data for this rOI
#         currentROIDF = aISDM.filter_based_on_lon_lat(firstDF,rOILonMin,rOILonMax,rOILatMin,rOILatMax)
#         currVessels = aISDM.get_list_of_unique_mmsi(currentROIDF)
        
        
#         '''
#         n44Sum[currentGrid] = n44Sum[currentGrid] + len(currVessels)
# #             print(currVessels)
#         #get the data for 2nd ROI
#         #useful for computing transition to the same box
#         nextROIDF = aISDM.filter_based_on_lon_lat(secondDF,rOILonMin,rOILonMax,rOILatMin,rOILatMax)
#         nextVessels = aISDM.get_list_of_unique_mmsi(nextROIDF)
# #             print(nextVessels)

#         #counting number of vessels in the same regeion
#         count44 = 0
#         for vesselA in currVessels:
#             for vesselB in nextVessels:
#                 if(vesselA == vesselB):
#                     count44 = count44 + 1            
# #             print(count44)

#         #update the corresponding matrix
#         n44HeatMap[currentGrid] = n44HeatMap[currentGrid] + count44

#         neihbLonMin, neihbLonMax, neihbLatMin, neihbLatMax = get_neighbour_lon_lat(currentGrid,0)

#         next40DF = aISDM.filter_based_on_lon_lat(secondDF,neihbLonMin,neihbLonMax,neihbLatMin,neihbLatMax)
#         next40Vessels = aISDM.get_list_of_unique_mmsi(next40DF)
# #             print(next40Vessels)
#         count40 = 0
#         for vesselA in currVessels:
#             for vesselB in next40Vessels:
#                 if(vesselA == vesselB):
#                     count40 = count40 + 1            
# #             print(count40)
#         n40HeatMap[currentGrid] = n40HeatMap[currentGrid] + count40

#         neihbLonMin, neihbLonMax, neihbLatMin, neihbLatMax = get_neighbour_lon_lat(currentGrid,1)
#         next41DF = aISDM.filter_based_on_lon_lat(secondDF,neihbLonMin,neihbLonMax,neihbLatMin,neihbLatMax)
#         next41Vessels = aISDM.get_list_of_unique_mmsi(next41DF)
# #             print(next41Vessels)
#         count41 = 0
#         for vesselA in currVessels:
#             for vesselB in next41Vessels:
#                 if(vesselA == vesselB):
#                     count41 = count41 + 1            
# #             print(count41)
#         n41HeatMap[currentGrid] = n41HeatMap[currentGrid] + count41

#         neihbLonMin, neihbLonMax, neihbLatMin, neihbLatMax = get_neighbour_lon_lat(currentGrid,2)
#         next42DF = aISDM.filter_based_on_lon_lat(secondDF,neihbLonMin,neihbLonMax,neihbLatMin,neihbLatMax)
#         next42Vessels = aISDM.get_list_of_unique_mmsi(next42DF)
# #             print(next42Vessels)
#         count42 = 0
#         for vesselA in currVessels:
#             for vesselB in next42Vessels:
#                 if(vesselA == vesselB):
#                     count42 = count42 + 1            
# #             print(count42)
#         n42HeatMap[currentGrid] = n42HeatMap[currentGrid] + count42

#         neihbLonMin, neihbLonMax, neihbLatMin, neihbLatMax = get_neighbour_lon_lat(currentGrid,3)
#         next43DF = aISDM.filter_based_on_lon_lat(secondDF,neihbLonMin,neihbLonMax,neihbLatMin,neihbLatMax)
#         next43Vessels = aISDM.get_list_of_unique_mmsi(next43DF)
# #             print(next43Vessels)
#         count43 = 0
#         for vesselA in currVessels:
#             for vesselB in next43Vessels:
#                 if(vesselA == vesselB):
#                     count43 = count43 + 1            
# #             print(count43)
#         n43HeatMap[currentGrid] = n43HeatMap[currentGrid] + count43

#         neihbLonMin, neihbLonMax, neihbLatMin, neihbLatMax = get_neighbour_lon_lat(currentGrid,5)
#         next45DF = aISDM.filter_based_on_lon_lat(secondDF,neihbLonMin,neihbLonMax,neihbLatMin,neihbLatMax)
#         next45Vessels = aISDM.get_list_of_unique_mmsi(next45DF)
# #             print(next45Vessels)
#         count45 = 0
#         for vesselA in currVessels:
#             for vesselB in next45Vessels:
#                 if(vesselA == vesselB):
#                     count45 = count45 + 1            
# #             print(count45)
#         n45HeatMap[currentGrid] = n45HeatMap[currentGrid] + count45

#         neihbLonMin, neihbLonMax, neihbLatMin, neihbLatMax = get_neighbour_lon_lat(currentGrid,6)
#         next46DF = aISDM.filter_based_on_lon_lat(secondDF,neihbLonMin,neihbLonMax,neihbLatMin,neihbLatMax)
#         next46Vessels = aISDM.get_list_of_unique_mmsi(next46DF)
# #             print(next46Vessels)
#         count46 = 0
#         for vesselA in currVessels:
#             for vesselB in next46Vessels:
#                 if(vesselA == vesselB):
#                     count46 = count46 + 1            
# #             print(count46)
#         n46HeatMap[currentGrid] = n46HeatMap[currentGrid] + count46

#         neihbLonMin, neihbLonMax, neihbLatMin, neihbLatMax = get_neighbour_lon_lat(currentGrid,7)
#         next47DF = aISDM.filter_based_on_lon_lat(secondDF,neihbLonMin,neihbLonMax,neihbLatMin,neihbLatMax)
#         next47Vessels = aISDM.get_list_of_unique_mmsi(next47DF)
# #             print(next47Vessels)
#         count47 = 0
#         for vesselA in currVessels:
#             for vesselB in next47Vessels:
#                 if(vesselA == vesselB):
#                     count47 = count47 + 1            
# #             print(count47)
#         n47HeatMap[currentGrid] = n47HeatMap[currentGrid] + count47

#         neihbLonMin, neihbLonMax, neihbLatMin, neihbLatMax = get_neighbour_lon_lat(currentGrid,8)
#         next48DF = aISDM.filter_based_on_lon_lat(secondDF,neihbLonMin,neihbLonMax,neihbLatMin,neihbLatMax)
#         next48Vessels = aISDM.get_list_of_unique_mmsi(next48DF)
# #             print(next48Vessels)
#         count48 = 0
#         for vesselA in currVessels:
#             for vesselB in next48Vessels:
#                 if(vesselA == vesselB):
#                     count48 = count48 + 1            
# #             print(count48)
#         n48HeatMap[currentGrid] = n48HeatMap[currentGrid] + count48
#         '''

In [ ]:
# StoreDir = "../Data/AIS_0117_1217_31_M120_345_M117/Transition/"
# opFile = StoreDir + '40.npy'
# np.save(opFile, n40HeatMap)

# opFile = StoreDir + '41.npy'
# np.save(opFile, n41HeatMap)

# opFile = StoreDir + '42.npy'
# np.save(opFile, n42HeatMap)

# opFile = StoreDir + '43.npy'
# np.save(opFile, n43HeatMap)

# opFile = StoreDir + '44.npy'
# np.save(opFile, n44HeatMap)

# opFile = StoreDir + '45.npy'
# np.save(opFile, n45HeatMap)

# opFile = StoreDir + '46.npy'
# np.save(opFile, n46HeatMap)

# opFile = StoreDir + '47.npy'
# np.save(opFile, n47HeatMap)

# opFile = StoreDir + '48.npy'
# np.save(opFile, n48HeatMap)

In [ ]:
# n40Disp = np.reshape(n40HeatMap,(yGrid.shape[0],xGrid.shape[0]))
# n40Disp = np.flip(n40Disp,axis=0)

# fig, ax = plt.subplots()

# ax.cla()
# ax.imshow(n40Disp,cmap=plt.get_cmap('hot'))

In [ ]:
# n41Disp = np.reshape(n41HeatMap,(yGrid.shape[0],xGrid.shape[0]))
# n41Disp = np.flip(n41Disp,axis=0)

# fig, ax = plt.subplots()

# ax.cla()
# ax.imshow(n41Disp,cmap=plt.get_cmap('hot'))

In [ ]:
# n42Disp = np.reshape(n42HeatMap,(yGrid.shape[0],xGrid.shape[0]))
# n42Disp = np.flip(n42Disp,axis=0)

# fig, ax = plt.subplots()

# ax.cla()
# ax.imshow(n42Disp,cmap=plt.get_cmap('hot'))

In [ ]:
# n43Disp = np.reshape(n43HeatMap,(yGrid.shape[0],xGrid.shape[0]))
# n43Disp = np.flip(n43Disp,axis=0)

# fig, ax = plt.subplots()

# ax.cla()
# ax.imshow(n43Disp,cmap=plt.get_cmap('hot'))

In [ ]:
# n44Disp = np.reshape(n44HeatMap,(yGrid.shape[0],xGrid.shape[0]))
# n44Disp = np.flip(n44Disp,axis=0)

# fig, ax = plt.subplots()

# ax.cla()
# ax.imshow(n44Disp,cmap=plt.get_cmap('hot'))

In [ ]:
# n45Disp = np.reshape(n45HeatMap,(yGrid.shape[0],xGrid.shape[0]))
# n45Disp = np.flip(n45Disp,axis=0)

# fig, ax = plt.subplots()

# ax.cla()
# ax.imshow(n45Disp,cmap=plt.get_cmap('hot'))

In [ ]:
# n46Disp = np.reshape(n46HeatMap,(yGrid.shape[0],xGrid.shape[0]))
# n46Disp = np.flip(n46Disp,axis=0)

# fig, ax = plt.subplots()

# ax.cla()
# ax.imshow(n46Disp,cmap=plt.get_cmap('hot'))

In [ ]:
# n47Disp = np.reshape(n47HeatMap,(yGrid.shape[0],xGrid.shape[0]))
# n47Disp = np.flip(n47Disp,axis=0)

# fig, ax = plt.subplots()

# ax.cla()
# ax.imshow(n47Disp,cmap=plt.get_cmap('hot'))

In [ ]:
# n48Disp = np.reshape(n48HeatMap,(yGrid.shape[0],xGrid.shape[0]))
# n48Disp = np.flip(n48Disp,axis=0)

# fig, ax = plt.subplots()

# ax.cla()
# ax.imshow(n48Disp,cmap=plt.get_cmap('hot'))